<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=89859025f33abbedbbe0995ba23e6599da665585b4a025abb4f4d3c30a2fd0aa
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-09 13:17:45
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 36.50 K (0.26%)
Current PnL: -20.79 L (-13.8%)
CY Booked + Current PnL: -9.35 L (-6.21%)
-------------------
Total profit:  1.60 L
Total loss:  -22.39 L
-------------------
Total Booked + Current PnL: 17.63 L (14.25%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.13%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.30 L (62.71%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.83%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.63,121317.0,20813.0,8298.0,2.36,20.71,6.84,28.96,79.0,2.51,0.87,61.71,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.60,40.0,M-SC,1.00,83382.0,-17395.0,17485.0,0.19,-17.26,20.97,0.09,245.0,-0.99,0.59,9.54,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.17,46.0,H-MC,3.03,188400.0,23730.0,19594.0,-0.44,14.41,10.40,26.31,89.0,1.21,1.34,23.54,X40N,NTT,AC
19,CIPLA,1662.74,-22.18,45.0,H-LC,3.67,207322.0,2822.0,20483.0,1.25,1.38,9.88,11.39,10.0,0.14,1.48,10.46,X40N,ATH,PHARMA
84,VOLTAS,1530.00,-0.94,53.0,H-MC,1.18,207765.0,16023.0,21732.0,1.30,8.36,10.46,19.69,99.0,0.74,1.48,15.72,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SONACOMS,1006.00,-40.66,48.0,M-SC,13.74,73850.0,-27309.0,102201.0,4.13,-27.00,138.39,74.03,202.0,-0.27,0.53,4.14,AR,BTT,AUTO
55,RAJOOENG,143.10,-40.48,45.0,H-SC,16.40,90230.0,-12270.0,52866.0,3.83,-11.97,58.59,39.61,114.0,-0.23,0.64,10.20,AR,ATH,MISC
32,HINDZINC,730.22,35.55,73.0,M-LC,7.33,220545.0,15469.0,97106.0,3.36,7.54,44.03,54.89,52.0,0.16,1.57,33.46,X5K,ATH,METALS
21,COFFEEDAY,80.00,-45.67,48.0,L-SC,29.38,80640.0,-32909.0,72963.0,3.27,-28.98,90.48,35.27,268.0,-0.45,0.58,97.37,XR,NTT,HOTELS
4,ANGELONE,3033.00,5.10,51.0,H-SC,11.24,186386.0,-4620.0,62327.0,2.50,-2.42,33.44,30.21,157.0,-0.07,1.33,17.96,X40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CERA,9475.00,-23.87,35.0,H-SC,1.69,138644.0,-37259.0,79277.0,-2.41,-21.18,57.18,23.89,149.0,-0.47,0.99,19.74,OX40N,NTT,CERAMICS
12,BATAINDIA,2096.00,-34.87,43.0,M-SC,4.41,97309.0,-31361.0,76660.0,-2.38,-24.37,78.78,35.20,219.0,-0.41,0.69,11.69,X40,NTT,FOOTWEAR
52,PGHH,17907.65,-27.37,56.0,H-MC,2.04,210270.0,9450.0,58350.0,-1.89,4.71,27.75,33.76,80.0,0.16,1.50,10.15,X40,ATH,FMCG
7,ATULAUTO,844.00,3770.27,57.0,M-SC,6.29,173740.0,-23392.0,113226.0,-1.69,-11.87,65.17,45.57,236.0,-0.21,1.24,23.18,XY24,NTT,AUTO
58,REPCOHOME,880.00,-59.02,58.0,H-SC,2.75,236076.0,-50429.0,307772.0,-1.64,-17.60,130.37,89.82,134.0,-0.16,1.68,24.90,XY24,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,61.0,H-SC,13.53,130086.0,-12393.0,31910.0,1.01,-8.70,24.53,13.70,163.0,-0.39,0.93,53.54,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.60,40.0,M-SC,1.00,83382.0,-17395.0,17485.0,0.19,-17.26,20.97,0.09,245.0,-0.99,0.59,9.54,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,47.0,H-SC,1.97,220536.0,-49131.0,85458.0,0.18,-18.22,38.75,13.47,138.0,-0.57,1.57,12.14,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-22.53,52.0,H-MC,6.54,104776.0,-27059.0,69288.0,1.00,-20.52,66.13,32.03,98.0,-0.39,0.75,17.36,OX40N,NTT,IT
18,CERA,9475.0,-23.87,35.0,H-SC,1.69,138644.0,-37259.0,79277.0,-2.41,-21.18,57.18,23.89,149.0,-0.47,0.99,19.74,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.63,121317.0,20813.0,8298.0,2.36,20.71,6.84,28.96,79.0,2.51,0.87,61.71,MH,ATH,METALS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,46.75,56.0,M-MC,12.57,231178.0,6216.0,161432.0,1.92,2.76,69.83,74.53,192.0,0.04,1.65,35.91,XY24,BTT,STEEL
32,HINDZINC,730.22,35.55,73.0,M-LC,7.33,220545.0,15469.0,97106.0,3.36,7.54,44.03,54.89,52.0,0.16,1.57,33.46,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.17,46.0,H-MC,3.03,188400.0,23730.0,19594.0,-0.44,14.41,10.40,26.31,89.0,1.21,1.34,23.54,X40N,NTT,AC
84,VOLTAS,1530.00,-0.94,53.0,H-MC,1.18,207765.0,16023.0,21732.0,1.30,8.36,10.46,19.69,99.0,0.74,1.48,15.72,XY25,NTT,AC
17,CAMS,5211.76,-6.05,46.0,H-SC,1.65,106842.0,4838.0,39083.0,0.94,4.74,36.58,43.06,122.0,0.12,0.76,22.67,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.63,121317.0,20813.0,8298.0,2.36,20.71,6.84,28.96,79.0,2.51,0.87,61.71,MH,ATH,METALS
32,HINDZINC,730.22,35.55,73.0,M-LC,7.33,220545.0,15469.0,97106.0,3.36,7.54,44.03,54.89,52.0,0.16,1.57,33.46,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,66.0,H-MC,11.50,193081.0,13273.0,99147.0,0.57,7.38,51.35,62.52,88.0,0.13,1.38,43.98,XR,NTT,BANKS
37,INDIAMART,4810.62,-54.08,45.0,H-SC,8.95,127995.0,4659.0,126971.0,-0.93,3.78,99.20,106.72,119.0,0.04,0.91,28.04,AR,ATH,MISC
86,WIPRO,420.00,-13.34,50.0,M-LC,5.66,152477.0,1532.0,107923.0,0.68,1.01,70.78,72.51,53.0,0.01,1.09,7.10,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.68,32.0,H-LC,5.63,205392.0,-46376.0,99595.0,0.24,-18.42,48.49,21.13,27.0,-0.47,1.46,10.48,X40,ATH,PAINTS
83,VBL,671.64,-22.30,33.0,H-LC,10.35,279986.0,-35856.0,148533.0,0.92,-11.35,53.05,35.67,5.0,-0.24,2.00,0.93,X40N,ATH,FMCG
1,ABB,7934.00,-41.33,39.0,H-LC,7.53,245328.0,-16291.0,135495.0,-0.34,-6.23,55.23,45.57,7.0,-0.12,1.75,4.39,AR,NTT,ELECTRICAL
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195527.0,-4611.0,25497.0,0.03,-2.30,13.04,10.44,4.0,-0.18,1.39,3.13,X40,NTT,FMCG
31,HINDUNILVR,2922.00,-10.50,41.0,H-LC,8.02,251180.0,-5883.0,41018.0,0.47,-2.29,16.33,13.67,24.0,-0.14,1.79,17.65,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-1.11,49.0,H-LC,1.94,158840.0,-27255.0,74734.0,-1.24,-14.65,47.05,25.51,15.0,-0.36,1.13,17.05,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.61,41.0,H-LC,2.27,195527.0,-4611.0,25497.0,0.03,-2.30,13.04,10.44,4.0,-0.18,1.39,3.13,X40,NTT,FMCG
48,LTIM,7230.2,-2.31,60.0,H-LC,2.54,242348.0,-3803.0,83004.0,0.81,-1.55,34.25,32.18,16.0,-0.05,1.73,33.94,X200,ATH,IT
51,NESTLEIND,1377.0,-10.26,51.0,H-LC,2.61,276120.0,10694.0,46084.0,-0.04,4.03,16.69,21.40,11.0,0.23,1.97,10.94,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.41,47.0,H-LC,3.09,214500.0,4134.0,24646.0,0.56,1.97,11.49,13.68,37.0,0.17,1.53,18.81,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.30,33.0,H-LC,10.35,279986.0,-35856.0,148533.0,0.92,-11.35,53.05,35.67,5.0,-0.24,2.00,0.93,X40N,ATH,FMCG
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195527.0,-4611.0,25497.0,0.03,-2.30,13.04,10.44,4.0,-0.18,1.39,3.13,X40,NTT,FMCG
1,ABB,7934.00,-41.33,39.0,H-LC,7.53,245328.0,-16291.0,135495.0,-0.34,-6.23,55.23,45.57,7.0,-0.12,1.75,4.39,AR,NTT,ELECTRICAL
74,TCS,4406.34,-28.36,52.0,H-LC,12.17,285440.0,-60516.0,128762.0,0.31,-17.49,45.11,19.73,1.0,-0.47,2.04,5.13,X40,ATH,IT
80,UNITDSPR,1644.00,-12.09,53.0,H-LC,6.66,230583.0,-3561.0,52181.0,0.56,-1.52,22.63,20.77,86.0,-0.07,1.64,5.29,X40N,NTT,BREWERIES


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-1.11,49.0,H-LC,1.94,158840.0,-27255.0,74734.0,-1.24,-14.65,47.05,25.51,15.0,-0.36,1.13,17.05,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195527.0,-4611.0,25497.0,0.03,-2.30,13.04,10.44,4.0,-0.18,1.39,3.13,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.68,32.0,H-LC,5.63,205392.0,-46376.0,99595.0,0.24,-18.42,48.49,21.13,27.0,-0.47,1.46,10.48,X40,ATH,PAINTS
19,CIPLA,1662.74,-22.18,45.0,H-LC,3.67,207322.0,2822.0,20483.0,1.25,1.38,9.88,11.39,10.0,0.14,1.48,10.46,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-14.41,47.0,H-LC,3.09,214500.0,4134.0,24646.0,0.56,1.97,11.49,13.68,37.0,0.17,1.53,18.81,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.36,52.0,H-LC,12.17,285440.0,-60516.0,128762.0,0.31,-17.49,45.11,19.73,1.0,-0.47,2.04,5.13,X40,ATH,IT
40,INFY,2275.00,-17.80,56.0,H-LC,7.90,320565.0,7677.0,164001.0,0.67,2.45,51.16,54.87,3.0,0.05,2.29,9.27,X40,BTT,IT
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195527.0,-4611.0,25497.0,0.03,-2.30,13.04,10.44,4.0,-0.18,1.39,3.13,X40,NTT,FMCG
83,VBL,671.64,-22.30,33.0,H-LC,10.35,279986.0,-35856.0,148533.0,0.92,-11.35,53.05,35.67,5.0,-0.24,2.00,0.93,X40N,ATH,FMCG
1,ABB,7934.00,-41.33,39.0,H-LC,7.53,245328.0,-16291.0,135495.0,-0.34,-6.23,55.23,45.57,7.0,-0.12,1.75,4.39,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,48.0,L-SC,29.38,80640.0,-32909.0,72963.0,3.27,-28.98,90.48,35.27,268.0,-0.45,0.58,97.37,XR,NTT,HOTELS
49,MASFIN,398.61,-20.96,39.0,H-SC,10.58,89940.0,-8040.0,29644.0,-1.38,-8.21,32.96,22.05,152.0,-0.27,0.64,30.73,XR,ATH,FINANCE
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.63,121317.0,20813.0,8298.0,2.36,20.71,6.84,28.96,79.0,2.51,0.87,61.71,MH,ATH,METALS
78,UJJIVANSFB,60.00,105.37,61.0,H-SC,13.53,130086.0,-12393.0,31910.0,1.01,-8.70,24.53,13.70,163.0,-0.39,0.93,53.54,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,14.41,145611.0,-3609.0,147795.0,1.70,-2.42,101.50,96.62,208.0,-0.02,1.04,57.58,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,14.41,145611.0,-3609.0,147795.0,1.70,-2.42,101.50,96.62,208.0,-0.02,1.04,57.58,XY25,NTT,FINANCE
32,HINDZINC,730.22,35.55,73.0,M-LC,7.33,220545.0,15469.0,97106.0,3.36,7.54,44.03,54.89,52.0,0.16,1.57,33.46,X5K,ATH,METALS
69,SURYODAY,240.00,64.47,63.0,H-SC,8.09,150623.0,-28448.0,94169.0,2.12,-15.89,62.52,36.71,135.0,-0.30,1.07,49.45,XR,NTT,BANKS
26,GREENPANEL,537.00,235.92,61.0,M-SC,4.93,149987.0,-33091.0,112070.0,0.03,-18.07,74.72,43.14,230.0,-0.30,1.07,41.08,XY24,NTT,MISC
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.63,121317.0,20813.0,8298.0,2.36,20.71,6.84,28.96,79.0,2.51,0.87,61.71,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.15
1,25,43.45
2,50,75.18


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.94
LC    32.27
MC    23.77
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.95
X40      14.64
X40N     11.20
XR       10.93
XY25     10.40
AR        8.15
OX40N     7.67
X200      1.73
MH        1.62
X5K       1.57
SR        1.12
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.90
H-LC    25.74
H-MC    20.48
M-SC    13.61
M-LC     5.52
M-MC     2.99
L-SC     1.43
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.72,-6.71,41.88
IT,12.89,-16.38,79.42
FINANCE,10.49,-14.67,70.05
MISC,6.91,-18.27,83.80
BANKS,6.34,-12.18,71.19
PAINTS,5.50,-19.61,37.44
ELECTRICAL,5.31,-10.44,49.19
INSURANCE,3.82,-3.38,42.02
AC,3.51,1.77,14.68


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3233833.0,22
XR,1377313.0,14
AR,1186552.0,9
X40,786659.0,10
X40N,664152.0,9
OX40N,570715.0,10
XY25,478878.0,7
SR,264892.0,2
X5K,97106.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3734815.0,29
M-SC,1522804.0,17
H-MC,1284107.0,15
H-LC,1184070.0,15
M-LC,376204.0,4
M-MC,364735.0,2
L-SC,260945.0,3
L-MC,60009.0,1
L-LC,42169.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1206535.0      6
M-SC       XY24       849871.0      7
H-SC       AR         808361.0      5
           XR         805017.0      7
H-MC       XY24       572713.0      4
H-LC       X40        489088.0      5
M-MC       XY24       364735.0      2
H-SC       X40N       306371.0      4
           OX40N      265183.0      4
           SR         264892.0      2
H-LC       X40N       221197.0      3
H-MC       X40        220911.0      4
H-LC       AR         210229.0      2
H-MC       XY25       177166.0      2
L-SC       XR         171860.0      2
M-LC       XY24       171175.0      2
M-SC       AR         167962.0      2
           XY25       147795.0      1
           OX40N      147159.0      4
H-MC       X40N       136584.0      2
M-SC       XR         133357.0      2
H-LC       XY25       111748.0      3
M-LC       XR         107923.0      1
H-MC       XR          99147.0      1
M-LC       X5K         97106.0      1
L-SC       OX40N       89085.0      1
H-LC       X200        83004.0      1
H-SC       MH          78456.0      1
M-SC       X40         76660.0      1
H-MC       OX40N       69288.0      1
H-LC       XY24        68804.0      1
L-MC       XR          60009.0      1
L-LC       XY25        42169.0      1
H-MC       MH           8298.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
